In [1]:
import pickle
import features
import utils
import openrouteservice
from ipyleaflet import Map, basemaps, Marker,AwesomeIcon, Popup,GeoJSON,LayersControl
from ipywidgets import HTML
from shapely.geometry import mapping
from ipywidgets import Layout
from datetime import timedelta

icon = AwesomeIcon(
    name='flag',
    marker_color='green',
    icon_color='white'
)

with open('route_simulation_results.pkl', 'rb') as f:
    route_simulation_results = pickle.load(f)   
obs=route_simulation_results[3]


center = [obs.place_centroid.geometry.y.iloc[0], obs.place_centroid.geometry.x.iloc[0]]
zoom = 14
label=HTML(obs.place_name)
popup = Popup(location=center, child=label, close_button=False, auto_close=False, close_on_escape_key=False)
marker = Marker(location=center, draggable=True,icon=icon,popup=popup,name=f"{obs.place_name} centroid")
layout = Layout(width="100%", height="500px")
m=Map(basemap=basemaps.OpenStreetMap.Mapnik, center=center, zoom=zoom,layout = layout)
m.add(marker)

tettsted_polygon = mapping(obs.polygon_place['geometry'][0])

layer = GeoJSON(
    data=tettsted_polygon,
    name = "tettsted polygon",
    style={
        "color": "blue",
        "weight": 2,
        "fillColor": "lightblue",
        "fillOpacity": 0.5,
        
    },
    hover_style={"fillColor": "yellow", "fillOpacity": 0.7},
)
iso_time = GeoJSON(
    data=obs.isochrone_time,
    name = f"isochrone_time {obs.max_driving_time_minutes} min",
    style={
        "color": "green",
        "weight": 2,
        "fillColor": "lightgreen",
        "fillOpacity": 0.5
    },
    hover_style={"fillColor": "yellow", "fillOpacity": 0.7},
)
iso_distance = GeoJSON(
    data=obs.isochrone_distance,
    name = f"isochrone_distance {obs.max_driving_distance_meters}",
    style={
        "color": "red",
        "weight": 2,
        "fillColor": "lightred",
        "fillOpacity": 0.5
    },
    hover_style={"fillColor": "yellow", "fillOpacity": 0.7},
)
addres_layer_iso_time =GeoJSON(data=utils.gdf_to_geojson(obs.subsampled_addresses_iso_time),point_style={
        "radius": 8,           # circle size
        "color": "green",       # outline color
        "fillColor": "yellow", # fill color
        "fillOpacity": 0.8,    # transparency
        "weight": 2            # outline width
    },name="addresses subsampled for iso time")
addres_layer_iso_distance =GeoJSON(data=utils.gdf_to_geojson(obs.subsampled_addresses_iso_distance),point_style={
        "radius": 8,           # circle size
        "color": "red",       # outline color
        "fillColor": "yellow", # fill color
        "fillOpacity": 0.8,    # transparency
        "weight": 2            # outline width
    },name="addresses subsampled for iso distance")
addres_layer_place =GeoJSON(data=utils.gdf_to_geojson(obs.subsampled_addresses_place),point_style={
        "radius": 8,           # circle size
        "color": "blue",       # outline color
        "fillColor": "yellow", # fill color
        "fillOpacity": 0.8,    # transparency
        "weight": 2            # outline width
    },name="addresses subsampled for iso distance")
m.add_layer(layer)
m.add_layer(iso_time)
m.add_layer(iso_distance)
m.add_layer(addres_layer_iso_time)
m.add_layer(addres_layer_iso_distance)
m.add_layer(addres_layer_place)

m.add_layer(LayersControl())

display(m)
popfraction=len(obs.subsampled_addresses_iso_time)/obs.addresses_in_isochrone_time
total_seconds = obs.summary_iso_time['duration']
dispatch_time = len(obs.stops_iso_time)*120
time_str = str(timedelta(seconds=total_seconds))
timetotal= str(timedelta(seconds=dispatch_time+total_seconds))
distance = obs.summary_iso_time['distance']/1000
print("SUMMARY FOR ISO TIME")
print(f"Traveled distance: {distance} km, traveled time: {time_str}, total time: {timetotal}")
print(f"fraction of population in the area: {popfraction*100}% from {obs.addresses_in_isochrone_time} addresses")

popfraction=len(obs.subsampled_addresses_iso_distance)/obs.addresses_in_isochrone_distance
total_seconds = obs.summary_iso_distance['duration']
dispatch_time = len(obs.stops_iso_distance)*120
time_str = str(timedelta(seconds=total_seconds))
timetotal= str(timedelta(seconds=dispatch_time+total_seconds))
distance = obs.summary_iso_distance['distance']/1000
print("SUMMARY FOR ISO DISTANCE")
print(f"Traveled distance: {distance} km, traveled time: {time_str}, total time: {timetotal}")
print(f"fraction of population in the area: {popfraction*100}% from {obs.addresses_in_isochrone_distance} addresses")

popfraction=len(obs.subsampled_addresses_place)/obs.addresses_in_place_polygon
total_seconds = obs.summary_place['duration']
dispatch_time = len(obs.stops_place)*120
time_str = str(timedelta(seconds=total_seconds))
timetotal= str(timedelta(seconds=dispatch_time+total_seconds))
distance = obs.summary_place['distance']/1000
print("SUMMARY FOR TETTSTED")
print(f"Traveled distance: {distance} km, traveled time: {time_str}, total time: {timetotal}")
print(f"fraction of population in the area: {popfraction*100}% from {obs.addresses_in_place_polygon} addresses")



Map(center=[np.float64(59.06826795), np.float64(11.41327458)], controls=(ZoomControl(options=['position', 'zoo…

SUMMARY FOR ISO TIME
Traveled distance: 39.1977 km, traveled time: 1:38:37.500000, total time: 2:28:37.500000
fraction of population in the area: 0.15075679913164083% from 16583 addresses
SUMMARY FOR ISO DISTANCE
Traveled distance: 46.499 km, traveled time: 1:28:00.600000, total time: 2:18:00.600000
fraction of population in the area: 0.14771048744460857% from 16925 addresses
SUMMARY FOR TETTSTED
Traveled distance: 6.0888 km, traveled time: 0:21:00.400000, total time: 1:11:00.400000
fraction of population in the area: 22.727272727272727% from 110 addresses
